<a href="https://colab.research.google.com/github/Smartsie/Playground/blob/master/Char_RNN_Thierry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
This code is just for fun to make a text generator by myself, inspired by Karpathy but completely made up by myself
T Martin, 12/11/2021
"""

'\nThis code is just for fun to make a text generator by myself, inspired by Karpathy but completely made up by myself\nT Martin, 12/11/2021\n'

# Import base libraries

In [5]:
import numpy as np
import pandas as pd
import base64
import requests
import tensorflow.keras as keras
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder

# Get Text data

In [6]:
def get_text(url):
  '''
    The function get_text will load a txt file at the specified url

    Args:
        url: The url where the text is located. 

    Returns:
        req!: a clean text
  '''

  req = requests.get(url)
  req = req.text
  return req

In [7]:
# Import the big text data
url = "http://f8cho.free.fr/RABELAIS.txt"


In [8]:
text=get_text(url)

In [ ]:
shakespeare_url='https://homl.info/shakespeare'
filepath=tf.keras.utils.get_file('shakespeare.txt',shakespeare_url)
with open(filepath) as f:
  shakespeare_text=f.read()
text=shakespeare_text

# Data preprocessing

In [9]:
tokenizer=keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(text)
max_id=len(tokenizer.word_index)
dataset_size=tokenizer.document_count
[encoded]=np.array(tokenizer.texts_to_sequences([text]))-1

In [10]:
train_size=dataset_size*90//100
dataset=tf.data.Dataset.from_tensor_slices(encoded[:train_size])
# Transform datecaset by using windows
n_steps=100
window_length=n_steps+1
dataset=dataset.window(window_length, shift=1,drop_remainder=True)
dataset=dataset.flat_map(lambda window: window.batch(window_length))
# Shuffle the dataset with a batch size
batch_size=32
dataset=dataset.shuffle(10000).batch(batch_size)
dataset=dataset.map(lambda windows: (windows[:,:-1],windows[:,1:]))
# One hot encoding
dataset=dataset.map(
    lambda X_batch,Y_batch: (tf.one_hot(X_batch,depth=max_id),Y_batch)
)
# prefetching
dataset=dataset.prefetch(1)

# Model definition

In [11]:
model=tf.keras.models.Sequential([
    tf.keras.layers.GRU(128,return_sequences=True,input_shape=[None,max_id],dropout=0.2,recurrent_dropout=0.2),
    tf.keras.layers.GRU(128,return_sequences=True,dropout=0.2,recurrent_dropout=0.2),
    tf.keras.layers.TimeDistributed(keras.layers.Dense(max_id,activation='softmax'))    
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam')


In [ ]:
model.fit(dataset,epochs=10,verbose=True)

Epoch 1/10
934/934 [==============================] - 462s 488ms/step - loss: 2.2393
Epoch 2/10
934/934 [==============================] - 457s 487ms/step - loss: 1.7193
Epoch 3/10
934/934 [==============================] - 456s 486ms/step - loss: 1.5395
Epoch 4/10
934/934 [==============================] - 457s 487ms/step - loss: 1.4356
Epoch 5/10
934/934 [==============================] - 459s 489ms/step - loss: 1.3658
Epoch 6/10
934/934 [==============================] - 464s 495ms/step - loss: 1.3139
Epoch 7/10
934/934 [==============================] - 455s 485ms/step - loss: 1.2736
Epoch 8/10
934/934 [==============================] - 451s 481ms/step - loss: 1.2413
Epoch 9/10
934/934 [==============================] - 447s 477ms/step - loss: 1.2134
Epoch 10/10
584/934 [=================>............] - ETA: 2:50 - loss: 1.2030

# Generate text

In [1]:
def preprocess(texts):
  X=np.array(tokenizer.texts_to_sequences(texts))-1
  return tf.one_hot(X,max_id)

In [2]:
def next_char(text,temperature=1):
  X_new=preprocess([text])
  y_proba=model.predict(X_new)[0,-1:,:]
  rescaled_logits=tf.math.log(y_proba)/temperature
  char_id=tf.random.categorical(rescaled_logits, num_samples=1)+1
  return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [3]:
def complete_text(text,n_chars=500,temperature=1):
  for _ in range(n_chars):
    text+=next_char(text,temperature)
  return text

In [4]:
result=complete_text("Pantagruel ",temperature=1)

NameError: ignored

In [ ]:
print(result)

Pantagruel cetture es de commes avact de pan la qu'il nonnêqr
